In [1]:
from pyspark.sql import SparkSession

In [2]:
# Spark 세션 생성
spark = SparkSession.builder \
    .appName("PMU Data Query") \
    .config("spark.jars", r"/home/sm/spark/postgresql-42.7.4.jar") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/10/08 17:08:03 WARN Utils: Your hostname, DESKTOP-F6CHKUO resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/10/08 17:08:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/08 17:08:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import json


with open("secret.json", "r") as f:
    secret = json.load(f)

In [4]:
# PostgreSQL에서 데이터 읽기

postgres_username = secret.get("POSTGRES_USER", "postgres")
postgres_password = secret.get("POSTGRES_PASSWORD", None)
postgres_port = int(secret.get("POSTGRES_PORT", 5432))
postgres_db = secret.get("POSTGRES_DB", "openpdc")
postgres_server = secret.get("POSTGRES_SERVER", "postgres-server")

jdbc_url = f"jdbc:postgresql://{postgres_server}:{postgres_port}/{postgres_db}"
connection_properties = {
    "user": postgres_username,
    "password": postgres_password,
    "driver": "org.postgresql.Driver"
}

In [48]:
sql = """

select *

from pmudata 

limit 500 

"""

In [63]:
postgreTable = spark.read.format("jdbc")\
                .option("url","jdbc:postgresql://{0}:{1}/{2}".format( postgres_server, postgres_port, postgres_db ) )\
                .option("driver", "org.postgresql.Driver")\
                .option("query", sql)\
                .option("user", postgres_username)\
                .option("password", postgres_password)\
                .load()

In [65]:
# Spark DataFrame을 Pandas on Spark DataFrame으로 변환
ps_df = postgreTable.pandas_api()

# 데이터 확인
ps_df

,signalid,key,timestamp,value
0,639ff1ca-383b-4e5f-956e-295827d5cd1a,PPA:498,2024-10-07 19:00:08.433,0.6457416415214539
1,5b18b7b7-0dcb-4293-a97a-2b66070b4375,PPA:501,2024-10-07 19:00:08.433,-110.75996633576005
2,1e276081-ba7b-4d4d-b448-56a351905af3,PPA:500,2024-10-07 19:00:08.433,0.5033420324325562
3,1091ed31-33d5-45f9-8473-2fb123971f81,PPA:503,2024-10-07 19:00:08.433,-110.79996392353965
4,0d38019c-3a8d-4f3b-91b7-9eeea0f1bafa,PPA:502,2024-10-07 19:00:08.433,0.5028038620948792
5,450ef9bc-cf4b-4459-8c5f-bd1210fc2c28,PPA:505,2024-10-07 19:00:08.433,-110.72997497511453
6,b0c1d83a-4623-4d07-a2ad-6878fcd98bed,PPA:504,2024-10-07 19:00:08.433,0.5032464861869812
7,c4dfea5f-4d4a-449c-89dd-892441f77982,PPA:507,2024-10-07 19:00:08.433,-110.72997497511453
8,36f487b0-ecd3-4569-945e-c9b0f04f06d3,PPA:506,2024-10-07 19:00:08.433,0.5020050406455994
9,d1d730c7-a9f0-42f6-804a-ee873e26270f,PPA:509,2024-10-07 19:00:08.433,-110.75997316594923
